## Project Problem

**Problem Statement**

A Chinese automobile company Geely Auto aspires to enter the US market by setting up their manufacturing unit there and producing cars locally to give competition to their US and European counterparts.

They have contracted an automobile consulting company to understand the factors on which the pricing of cars depends. Specifically, they want to understand the factors affecting the pricing of cars in the American market, since those may be very different from the Chinese market. The company wants to know:

- Which variables are significant in predicting the price of a car
- How well those variables describe the price of a car

Based on various market surveys, the consulting firm has gathered a large dataset of different types of cars across the Americal market.

**Business Goal**

We are required to model the price of cars with the available independent variables. It will be used by the management to understand how exactly the prices vary with the independent variables. They can accordingly manipulate the design of the cars, the business strategy etc. to meet certain price levels. Further, themodel will be a good way for management to understand the pricing dynamics of a new market.




Project Objective :
* Based on the data and data dictionary, We have prediction / regression problem.
* We wil make prediction on the target variable `PRICE`
* And we will build a model to get best prediction on the price variable. (For that we will use RMSE(Root Mean Squared Error) and R square statistic)



Data Source : https://archive.ics.uci.edu/ml/datasets/Automobile

## Importing Libraries

In [128]:
#fundamental data libraries
import numpy as np
import pandas as pd

#visualisation libararies
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

In [129]:
pd.set_option('max_columns',100)
pd.set_option('max_rows',900)

data=pd.read_csv('../input/car-data/CarPrice_Assignment.csv')
data.head()

## Understanding Data

DATA DICTONARY

1. Car_ID: Unique id of each observation
2. Symboling: Its assigned insurance risk rating, A value of +3 indicates that the auto is risky, -3 that it is probably pretty safe.
3. carCompany: Name of car company
4. fueltype: Car fuel type i.e gas or diesel
5. aspiration: Aspiration used in a car
6. doornumber: Number of doors in a car
7. carbody: body of car
8. drivewheel: type of drive wheel
9. enginelocation: Location of car engine
10. wheelbase: Weelbase of car 
11. carlength: Length of car
12. carwidth: Width of car
13. carheight: height of car
14. curbweight: The weight of a car without occupants or baggage.
15. enginetype: Type of engine.
16. cylindernumber: cylinder placed in the car
17. enginesize: Size of car
18. fuelsystem: Fuel system of car
19. boreratio: Boreratio of car
20. stroke: Stroke or volume inside the engine
21. compressionratio: compression ratio of car
22. horsepower: Horsepower
23. peakrpm: car peak rpm
24. citympg: Mileage in city
25. highwaympg: Mileage on highway
26. price: Price of car

In [130]:
print("The number of rows are = {0} \nThe number of columns are = {1}".format(data.shape[0],data.shape[1]))

In [131]:
data.info()

Descriptive Statistics of Numerical Features

In [132]:
data.describe()

Descriptive Statistics of Categorical Features

In [133]:
data.describe(include=['object'])

Checking for Missing Values

In [134]:
def missing(df):
    missing_number = df.isnull().sum().sort_values(ascending=False)
    missing_percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_values = pd.concat([missing_number, missing_percent], axis=1, keys=['Number of Missing values', 'Percentage of Missing values'])
    return missing_values

missing(data)

Dataset contains no missing values

## Data Handling

Here we will perform the following tasks:

1. Remove `car_ID` as we already have an index
2. Split `CarName` into 2 features `Company` and `Model` and keep only Company since Model alone does not tell any information and Company takes more preferance in determining price of a car.
3. Check unique values in different features and clean them if required

In [135]:
#removing car_ID
data.drop(['car_ID'],axis=1,inplace=True)

#Splitting CarName
data['CarCompany']=data.CarName.apply(lambda x: x.split(' ')[0])
data.drop(['CarName'],axis=1,inplace=True)

In [136]:
data.head()

In [137]:
data.CarCompany.unique()

Fixing Spelling Errors in `CarCompany` feature:

* `maxda` should be renamed to `mazda`
* `Nissan` should be renamed to `nissan`
* `porcshce` should be renamed to `porsche`
* `toyouta` should be renamed to `toyota`
* `vokswagen` and `vw` should be renamed to `volkswagen`  

In [138]:
data.CarCompany.replace({'maxda':'mazda',
                        'Nissan':'nissan',
                        'porcshce':'porsche',
                        'toyouta':'toyota',
                        'vokswagen':'volkswagen',
                        'vw':'volkswagen'}, inplace=True);
data.CarCompany.unique()

Lets see unique values in all columns 

In [139]:
def unique_values_fn(df):
    for col in df:
        print(col)
        print(df[col].unique())
        print("\n")

unique_values_fn(data)

Data is clean and does not contain irrelevant values

## EDA and Visualisation

Q: How does the distribution of the target variable look like?

In [140]:
np.quantile(data.price,0.25)

In [141]:
plt.figure(figsize=(12,15))

plt.subplot(2,1,1)
sns.histplot(data.price,kde=True,color='orange',line_kws={'lw': 3,'label':'kde approximation'},edgecolor=None)
plt.axvline(x=np.quantile(data.price,0.25),color='blue',label='Q1')
plt.axvline(x=np.quantile(data.price,0.5),color='green',label='Median (Q2)')
plt.axvline(x=np.quantile(data.price,0.75),color='red',label='Q3')
plt.axvline(x=np.mean(data.price),color='black',label='Mean')
plt.title("Distribution of Car Prices")
plt.xlabel("Price",fontsize=10)
plt.ylabel("")
plt.legend()

plt.subplot(2,1,2)
sns.boxplot(y=data.price,color='orange')
plt.title("Spread of Car Prices")
plt.ylabel("Price",fontsize=10)

plt.tight_layout(pad=5);

In [142]:
data.price.describe(percentiles=[0.25,0.5,0.75,0.85,0.9])

In [143]:
np.var(data.price)

From the above data we observe:

1. Price is positively skewed (since tail is on the right and mode<median<mean )
2. There is big difference between mean and median
3. Variance of price is large(= 63510435) which can also be seen from percentiles - 85% of data is less than 18,500 but rest 15% of data is between 18,500-45400
4. From the boxplot we see there are some outliers however since dataset is very small we might leave them as is.
5. Average Price of car in dataset is 13,276.71 $

Q: What is the distribution of the Categorical Variables and how do they relate to the target variable?

Categorical Variables:
- CarCompany
- Symboling
- fueltype
- enginetype
- carbody
- doornumber
- enginelocation
- fuelsystem
- cylindernumber
- aspiration
- drivewheel

In [144]:
#figure
plt.figure(figsize=(30, 25))
plt.suptitle("Distribution Study of Categorical variables",fontsize=30,y=1.08)
plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.3, wspace=0.4)

#plot1
plt.subplot(2,3,1)
plt1 = data.CarCompany.value_counts().plot(kind='barh')
plt.title('Companies Histogram')
plt1.set(ylabel = 'Car company', xlabel='Frequency of company')

#plot2
plt.subplot(2,3,2)
plt1 = data.fueltype.value_counts().plot(kind='bar')
plt.title('Fuel Type Histogram')
plt1.set(xlabel = 'Fuel Type', ylabel='Frequency of fuel type')

#plot3
plt.subplot(2,3,3)
plt1 = data.carbody.value_counts().plot(kind='bar')
plt.title('Car Type Histogram')
plt1.set(xlabel = 'Body Type', ylabel='Frequency of Car type')

#plot4
plt.subplot(2,3,4)
df = data.groupby(['CarCompany'])['price'].mean().sort_values(ascending = False)
plt1=df.plot.barh()
plt.title('Company Name vs Average Price')
plt1.set(xlabel = 'Average Car Price of Company', ylabel='Car Company')

#plot5
plt.subplot(2,3,5)
df = data.groupby(['fueltype'])['price'].mean().sort_values(ascending = False)
plt1=df.plot.bar()
plt.title('Fuel Type vs Average Price')
plt1.set(xlabel = 'Fuel Type', ylabel='Average Price')

#plot6
plt.subplot(2,3,6)
df = data.groupby(['carbody'])['price'].mean().sort_values(ascending = False)
plt1=df.plot.bar()
plt.title('Car Type vs Average Price')
plt1.set(xlabel = 'Body Type', ylabel='Average Price');

In [145]:
data.fueltype.value_counts()

Observations:
- Toyota seemed to be favored car company.
- Number of gas fueled cars are way more than diesel - 185 to 20.
- Sedan is the top car type prefered.
- Jaguar and Buick seem to have highest average price.
- diesel has higher average price than gas.
- hardtop and convertible have higher average price.

Company vs Price Distribution

In [146]:
plt.figure(figsize=(20,8))

sns.swarmplot(x='CarCompany',y='price',data=data)
plt.title("Company vs Price Relationship")
plt.xlabel("Car Company",fontsize=15,labelpad=6)
plt.ylabel("Price",fontsize=10,labelpad=6)
plt.axhline(y=np.mean(data.price),label='Average Price',alpha=0.5)
plt.legend()
plt.grid(True);

Inference: 

- Luxury Companies such as BMW, Volvo,Porsche,Jaguar have high priced cars as expected.
- Buick seems to be an odd one out here as although its known for SUVs it is not a particularly luxury brand. Our ML model will overestimate values of Buick car brand if we train it on this data. However this requires more subtle understanding of the Domain, more precisely the American Car Market so we let it be for now.


Q: Is it true that cheap cars are not safe and vice versa?

*Safety rating of a Car is determined by its Symboling rating. Symboling corresponds to the degree to which the auto is more risky than its price indicates. Cars are initially assigned a risk factor symbol associated with its price. Then, if it is more risky (or less), this symbol is adjusted by moving it up (or down) the scale. A value of +3 indicates that the auto is risky, -3 that it is probably pretty safe. This term comes from Acturial statistics*

In [147]:
plt.figure(figsize=(20,8))
plt.suptitle("Relationship between Car's Safety(as measured by its Symboling rating) and Price",fontsize=20,y=1.03)

plt.subplot(1,2,1)
plt.title('Symboling Histogram')
sns.countplot(x=data.symboling, palette=("cubehelix"))
plt.xlabel("Symboling Rating")

plt.subplot(1,2,2)
plt.title('Symboling Rating vs Price')
sns.boxplot(x=data.symboling, y=data.price, palette=("cubehelix"))
plt.xlabel("Symboling Rating")
plt.ylabel("Price");

Inference:
- It seems that the symboling with 0 and 1 values  are most sold, which probably mean we have more of the usual non-sport cars.
- The cars with -1 symboling seems to be high priced (as it makes sense too, insurance risk rating -1 is quite good). But it seems that symboling with 3 value has the price range similar to -2 value. There is a dip in price at symboling 1.

Lets look at cars in -2 and +3 categories to understand why their prices are so close.

In [148]:
#cars with very high safety rating(-2)
data.loc[data.symboling==-2]

All of these are Volvos which puts the safety ratings in perspective since Volvo is known to make the safest and durable cars on the planet. Volvos are also not cheap which explains why the prices of these -2 rating cars is near to those of super cars in 3 categories.

In [149]:
#cars with very low safety ratings(+3)
data.loc[data.symboling==3].sample(7)

In [150]:
avg=np.round(data.loc[data.symboling==3].horsepower.mean(),2)
quant=np.round(np.count_nonzero(np.array(data.horsepower)<avg) / data.horsepower.size,3)
print("Average Horsepower of +3 Symboling rated Cars : {0} which is {1} quantile of Horsepower in the dataset".format(avg,quant))

All these cars are high end sport cars from buick, nissan, porsche, alfa-romero so that explains their price and their riskyness.Also the average horsepower of this class is quite high which also is in line with our reasoning that these are mostly sport cars.

Engine Types and their relationship with Price



In [151]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,1)
plt.title('Engine Type Histogram')
sns.countplot(x=data.enginetype, palette=("Blues_d"))

plt.subplot(1,2,2)
plt.title('Engine Type vs Price')
sns.boxplot(x=data.enginetype, y=data.price, palette=("PuBuGn"))

df = pd.DataFrame(data.groupby(['enginetype'])['price'].mean().sort_values(ascending = False))
df.plot.bar(figsize=(8,6))
plt.title('Engine Type vs Average Price');

In [152]:
data.loc[data.enginetype=='dohcv']

Inferences:

- ohc Engine type seems to be most favored type.
- ohcv has the highest price range (While dohcv has only one row), ohc and ohcf have the low price range.

Visualising Numerical features relationship with target variable

In [153]:
num_cols=['carlength','wheelbase','carwidth','carheight','curbweight','enginesize','boreratio','stroke','compressionratio','horsepower','peakrpm','citympg','highwaympg']


plt.figure(figsize=(15,35))
plt.suptitle("Numerical Features and Price Relationship",fontsize=25,y=1.03)
plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=0.3, wspace=0.4)

def printer_fn():
    for i,var in enumerate(num_cols):
        plt.subplot(8,2,i+1)
        sns.regplot(x=var,y='price',data=data)
        plt.title(var+' vs Price')
        plt.ylabel('Price')
        plt.xlabel(var)
        
     

printer_fn()

Inference:
- `carwidth`, `carlength` and `curbweight` seems to have a poitive correlation with `price` and are roughly linear.
- `carheight` doesn't show any significant trend with `price`.
- `curbweight` shows high correlation with `price` and also have very linear pattern with positive correlation
- `enginesize`, `boreratio`, `horsepower`, `wheelbase` - seem to have a significant positive correlation with `price`.
- `citympg`, `highwaympg` - seem to have a significant negative correlation with `price` which makes sense as expensive luxury cars don't have good mileage .
- `peakrpm` and `stroke` don't have much correlation with `price`.

## Feature Enginnering

We will do the following enginnering tasks to prepare the data for modelling,
1. Combine `citympg` and `highwaympg` into one feature `fueleconomy`

    (To calculate combined Mileage, manufacturers assign a percentage value to city and highway mileage. City mileage is        valued at 55% and highway mileage is valued at 45%. These two numbers are combined to give you the combined mileage.)

2. Create another categorical variable `carrange` by binning car `price` in three bins : 'budget','medium','highend'. This variable will be highly correlated to `price` and also with other features hence will explain variability in y.


In [154]:
#Fuel economy
data['fueleconomy'] = (0.55 * data['citympg']) + (0.45 * data['highwaympg'])

def binning_fn(val):
    if val<10000:
        return('Budget')
    elif val<20000 and val>10000:
        return('Medium')
    else:
        return('Highend')

data['CarRange']=data.price.apply(binning_fn)

In [155]:
data.head()

List of significant variables after Visual analysis :
- Car Range 
- Engine Type 
- Fuel type 
- Car Body 
- Aspiration 
- Cylinder Number 
- Drivewheel 
- Curbweight 
- Car Length
- Car width
- Engine Size 
- Boreratio 
- Horse Power 
- Wheel base 
- Fuel Economy 

In [156]:
#making a new copy of dataframe with important features only
cars_lr = data[['price', 'fueltype', 'aspiration','carbody', 'drivewheel','wheelbase',
                  'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower', 
                    'fueleconomy', 'carlength','carwidth', 'CarRange']]
cars_lr.head()

In [157]:
sns.pairplot(cars_lr, palette="Set2", diag_kind="kde", height=2.5);

Encoding Categorical Variables


Since we wish to apply Linear Regression Model on this data we need to be aware of 'Dummy Variable Trap' where dummy variables are mathematically related by the expression, sum of all entires=1 ,so we remove one column to not include Multicollinearity in dataset.

In [158]:
cars_lr=pd.get_dummies(cars_lr,columns=['carbody','fueltype','aspiration','drivewheel','enginetype','cylindernumber','CarRange'],drop_first=True)

In [159]:
cars_lr.head()

In [160]:
cars_lr.shape

## Correlation Analysis

In [161]:
#Correlation using heatmap
plt.figure(figsize = (30, 25))
sns.heatmap(cars_lr.corr(), annot = True, cmap="YlGnBu");

Observations:

- First 9 variables are highly correlated with each other so we can expect to see multicollinearity in our data. 
- These 9 are also most correlated with target variable so we can expect them to be more significant in prediction that others.

In [162]:
from sklearn.model_selection import train_test_split

np.random.seed(42)

X_train,X_test,y_train,y_test = train_test_split(cars_lr.drop(['price'],axis=1), cars_lr.price, test_size = 0.2)

In [163]:
X_train.head()

In [164]:
y_train.head()

Scaling Training data. 

In regression, it is often recommended to scale the features so that the predictors have a mean of 0. This makes it easier to interpret the intercept term as the expected value of Y when the predictor values are set to their means. Scaling has no effect on OLS of Linear Regression Model estimates or inferences drawn on them.

In [165]:
from sklearn.preprocessing import StandardScaler

num_vars = ['wheelbase', 'curbweight', 'enginesize', 'boreratio', 'horsepower','fueleconomy','carlength','carwidth']
scaler1 = StandardScaler().fit(X_train[num_vars])
X_train[num_vars] = scaler1.transform(X_train[num_vars])

scaler2=StandardScaler().fit(np.asarray(y_train).reshape(-1,1))
y_train=scaler2.fit_transform(np.asarray(y_train).reshape(-1,1))

y_train=pd.Series(y_train.ravel())


In [166]:
X_train.head()

In [168]:
y_train.head()

In [169]:
X_train.shape

## Model Building

Since we have 30 features in X train after picking best looking features from EDA analysis we still need to find smaller subset of these features that have the most effect on Y. From Correlation heatmap we saw that there are some features that have higher correlation value with y than other so we will use Recursive Feature Extraction(a backward elimination technique) that will find best k features we want. We will take k=10 and then apply further statistical analysis to prune the subset even further. 

In [185]:
from sklearn.feature_selection import RFE

estimator=LinearRegression()
selector = RFE(estimator,n_features_to_select=15, step=1)
selector = selector.fit(X_train, y_train)
selector.get_feature_names_out()

In [218]:
df=pd.DataFrame(zip(selector.ranking_,X_train.columns),columns=['ranking','feature_name'])
df.set_index(['ranking']).sort_index()

We will now build a baseline model from these features and then apply statisticial analysis on that to improve it.

In [220]:
X_train_rfe=X_train[selector.get_feature_names_out()]
X_train_rfe.head()

In [ ]:
import statsmodels.api as sm 
from statsmodels.stats.outliers_influence import variance_inflation_factor

def build_model(X,y):
    '''
    function that builds a linear regression model based on design matrix , X and y values. Prints regression summary and return the new design matrix.
    '''
    X = sm.add_constant(X) #Adding the constant column in design matrix
    lm = sm.OLS(y,X).fit() # fitting the model
    print(lm.summary()) # model summary
    return X
  
def checkVIF(X):
     '''
    function that calculates VIF of each feature in the input design matrix,X
    '''
    vif = pd.DataFrame()
    vif['Features'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return(vif)
    

## Model 1 

In [ ]:
from stats